In [ ]:
import sys
sys.path.append("..")
from common_utils import set_data_home
set_data_home("/mnt/d")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset, join

sd = Sales_Dataset(join(DATA_HOME, "sales_data"))
len(sd)

In [ ]:
# print(sd.O.describe())
# print(sd.TR.describe())
# print(sd.TS.describe())
# print(sd.S.describe())

In [ ]:
# print(sd.TR[sd.TR.index == (1, "2014-01-01")])
for i in range(len(sd)):
    test = sd[i]
    break

print(test)